# Asylum and resettlement datasets 

Analysis of asylum and resettlement datasets in the UK.

## 1. Asylum applications, decisions and resettlement

First, we will import the dataset.

In [275]:
import pandas as pd

df = pd.read_csv('./data/asylum_decisions.csv', dtype={"Year":str, "Decisions": str})

In [276]:
df.head()

,Year,Quarter,Nationality,Region,Case type,Case outcome group,Case outcome,Age,Sex,Applicant type,UASC,Decisions
0,2001,2001 Q1,Afghanistan,Asia Central,Asylum Case,Grant of Protection,Asylum,Total (pre-2009),Total (pre-2009),Main Applicant,Total (pre-2006),809
1,2001,2001 Q1,Afghanistan,Asia Central,Asylum Case,Grant of Other Leave,Exceptional Leave to Remain,Total (pre-2009),Total (pre-2009),Main Applicant,Total (pre-2006),"2,652"
2,2001,2001 Q1,Afghanistan,Asia Central,Asylum Case,Refused,Certified Refusal,Total (pre-2009),Total (pre-2009),Main Applicant,Total (pre-2006),88
3,2001,2001 Q1,Afghanistan,Asia Central,Asylum Case,Refused,Other Refusals,Total (pre-2009),Total (pre-2009),Main Applicant,Total (pre-2006),301
4,2001,2001 Q1,Afghanistan,Asia Central,Asylum Case,Refused,3rd Country Refusal,Total (pre-2009),Total (pre-2009),Main Applicant,Total (pre-2006),15


In [277]:
# Clean decisions column
df['Decisions'] = df['Decisions'].str.replace(',', '')
df['Decisions'] = df['Decisions'].astype(float)

**Question:** Have asylum decision trends change over time?

In [278]:
df_q1 = df[['Year', 'Nationality', 'Case outcome group', 'Decisions']].groupby(by=['Year', 'Nationality', 'Case outcome group']).sum()
df_q1.reset_index(level=0, inplace=True)
df_q1 = df_q1.sort_values('Decisions', ascending=False)
df_q1 = df_q1.reset_index()
df_q1

,Nationality,Case outcome group,Year,Decisions
0,Serbia and Montenegro,Refused,2001,9011.0
1,Iraq,Grant of Other Leave,2002,8779.0
2,Sri Lanka,Refused,2001,8371.0
3,Afghanistan,Grant of Other Leave,2001,7498.0
4,Turkey,Refused,2001,7340.0
...,...,...,...,...
8181,Cuba,Grant of Protection,2006,1.0
8182,Cyprus,Withdrawn,2006,1.0
8183,Refugee,Grant of Other Leave,2015,1.0
8184,Qatar,Grant of Protection,2015,1.0


In [279]:
df_q1.loc[0:1]

,Nationality,Case outcome group,Year,Decisions
0,Serbia and Montenegro,Refused,2001,9011.0
1,Iraq,Grant of Other Leave,2002,8779.0


In [280]:
nationalities_filter = ['Serbia and Montenegro', 'Syria', 'Iran', 'Morocco', 'Iraq', 'Pakistan', 'Eritrea', 'Nigeria']

In [281]:
df_q1_ref = df_q1.loc[(df_q1['Case outcome group'] == 'Refused') | (df_q1['Case outcome group'] == 'Withdrawn')]
df_q1_ref = df_q1_ref[df_q1_ref['Nationality'].isin(nationalities_filter)]

df_q1_ref = df_q1_ref[['Year', 'Nationality', 'Decisions']].groupby(by=['Year', 'Nationality']).sum()
df_q1_ref.reset_index(level=0, inplace=True)
df_q1_ref = df_q1_ref.sort_values('Decisions', ascending=False)
df_q1_ref = df_q1_ref.reset_index()
df_q1_ref['Rejected'] = df_q1_ref['Decisions']
df_q1_ref = df_q1_ref.drop(['Decisions'], axis=1)

In [282]:
df_q1_ref.head()

,Nationality,Year,Rejected
0,Serbia and Montenegro,2001,9258.0
1,Iraq,2001,6377.0
2,Iran,2001,5201.0
3,Iraq,2004,4965.0
4,Iraq,2003,4933.0


In [283]:
df_q1_not_ref = df_q1.loc[(df_q1['Case outcome group'] != 'Refused') & (df_q1['Case outcome group'] != 'Withdrawn')]
df_q1_not_ref = df_q1_not_ref[df_q1_not_ref['Nationality'].isin(nationalities_filter)]


df_q1_not_ref = df_q1_not_ref[['Year', 'Nationality', 'Decisions']].groupby(by=['Year', 'Nationality']).sum()
df_q1_not_ref.reset_index(level=0, inplace=True)
df_q1_not_ref = df_q1_not_ref.sort_values('Decisions', ascending=False)
df_q1_not_ref = df_q1_not_ref.reset_index()
df_q1_not_ref['Accepted'] = df_q1_not_ref['Decisions']
df_q1_not_ref = df_q1_not_ref.drop(['Decisions'], axis=1)

In [284]:
df_totals = df_q1_ref.merge(df_q1_not_ref, on =['Nationality', 'Year'])

In [285]:
df_totals.columns

Index(['Nationality', 'Year', 'Rejected', 'Accepted'], dtype='object')

In [286]:
df_totals['Ratio'] = df_totals['Rejected']/df_totals['Accepted']

In [287]:
df_totals

,Nationality,Year,Rejected,Accepted,Ratio
0,Serbia and Montenegro,2001,9258.0,2420.0,3.825620
1,Iraq,2001,6377.0,2718.0,2.346210
2,Iran,2001,5201.0,880.0,5.910227
3,Iraq,2004,4965.0,202.0,24.579208
4,Iraq,2003,4933.0,2412.0,2.045191
...,...,...,...,...,...
140,Morocco,2003,37.0,5.0,7.400000
141,Morocco,2004,34.0,7.0,4.857143
142,Morocco,2005,34.0,8.0,4.250000
143,Morocco,2007,31.0,1.0,31.000000


In [288]:
import matplotlib.pyplot as plt
import plotly.express as px



df_totals['Year'] = df_totals['Year'].astype(int)
df_totals = df_totals.sort_values('Year', ascending=False)

fig = px.bar(df_totals, x="Year", y="Ratio", text='Ratio', color = 'Nationality', labels={
                     "year": "Year",
                     "Ratio": "Rate"
                 },)
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',   title={
        'text': "Rate between Rejections and Approvals on Asylum Applications ",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_traces(texttemplate='%{text:.3s}')
fig.update_layout(uniformtext_minsize=25)
fig.show()

In [190]:
df_totals

,Nationality,Case outcome group,Year,Decisions,Accepted,ratio,Ratio
256,Syria,Withdrawn,2020,0.0,0.0,NaN,NaN
248,Morocco,Refused,2020,0.0,0.0,NaN,NaN
257,Syria,Refused,2020,0.0,0.0,NaN,NaN
255,Iraq,Refused,2020,0.0,0.0,NaN,NaN
253,Nigeria,Refused,2020,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...
2,Iraq,Refused,2001,6317.0,2718.0,2.324135,2.324135
187,Syria,Withdrawn,2001,5.0,35.0,0.142857,0.142857
119,Eritrea,Withdrawn,2001,7.0,351.0,0.019943,0.019943
118,Eritrea,Refused,2001,954.0,351.0,2.717949,2.717949


In [230]:
df.loc[(df['Nationality'] == 'Nigeria') & (df['Year'] == '2020')]

,Year,Quarter,Nationality,Region,Case type,Case outcome group,Case outcome,Age,Sex,Applicant type,UASC,Decisions
87649,2020,2020 Q1,Nigeria,Africa Sub-Saharan,Asylum Case,Refused,Other Refusals,30-49,Male,Main applicant,Non-UASC,NaN
87650,2020,2020 Q1,Nigeria,Africa Sub-Saharan,Asylum Case,Refused,Other Refusals,30-49,Female,Main applicant,Non-UASC,NaN
87651,2020,2020 Q1,Nigeria,Africa Sub-Saharan,Asylum Case,Refused,Other Refusals,18-29,Male,Main applicant,Non-UASC,NaN
87652,2020,2020 Q1,Nigeria,Africa Sub-Saharan,Asylum Case,Refused,Other Refusals,50-69,Female,Main applicant,Non-UASC,NaN
87653,2020,2020 Q1,Nigeria,Africa Sub-Saharan,Asylum Case,Refused,Certified Refusal,30-49,Male,Main applicant,Non-UASC,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
91287,2020,2020 Q4,Nigeria,Africa Sub-Saharan,Asylum Case,Withdrawn,Non-Substantiated Withdrawal,18-29,Male,Main Applicant,Non-UASC,NaN
91288,2020,2020 Q4,Nigeria,Africa Sub-Saharan,Asylum Case,Withdrawn,Non-Substantiated Withdrawal,30-49,Male,Main Applicant,Non-UASC,NaN
91289,2020,2020 Q4,Nigeria,Africa Sub-Saharan,Asylum Case,Withdrawn,Other Withdrawal,18-29,Male,Main Applicant,Non-UASC,NaN
91290,2020,2020 Q4,Nigeria,Africa Sub-Saharan,Asylum Case,Withdrawn,Other Withdrawal,30-49,Female,Main Applicant,Non-UASC,NaN
